<a href="https://colab.research.google.com/github/VittorioBartolomeoSecondin/DVIS-CAValli_Team/blob/main/Temperatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data exploration and preprocessing

## Importing libraries and connecting to Google Drive

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import re
import json
import csv
import requests
import os
import zipfile
from google.colab import files
from google.colab import drive
drive.mount('/content/drive') # turned out to be a good workaround to load a huge amount of data and keep it available

Mounted at /content/drive


## Reading the data

In [3]:
all_files = glob.glob("/content/drive/MyDrive/tree_dataset/final_dataset/*.txt")
all_files.sort()
print(all_files)

['/content/drive/MyDrive/tree_dataset/final_dataset/climdiv-tmaxst-v1.0.0-20231106.txt', '/content/drive/MyDrive/tree_dataset/final_dataset/climdiv-tminst-v1.0.0-20231106.txt', '/content/drive/MyDrive/tree_dataset/final_dataset/climdiv-tmpcst-v1.0.0-20231106.txt']


In [4]:
column_names = ['Code', 'JanF', 'FebF', 'MarF', 'AprF', 'MayF', 'JunF', 'JulF', 'AugF', 'SepF', 'OctF', 'NovF', 'DecF']
column_data_types = {column_names[0]: str}
column_data_types.update({column_names[i]: float for i in range(1, 13)})
max_dataset = pd.read_csv(all_files[0], delimiter = r'\s+', header = None, names = column_names, dtype = column_data_types)
min_dataset = pd.read_csv(all_files[1], delimiter = r'\s+', header = None, names = column_names, dtype = column_data_types)
avg_dataset = pd.read_csv(all_files[2], delimiter = r'\s+', header = None, names = column_names, dtype = column_data_types)

## Preparing the datasets for the export

Replace -99.9 with NULL values

In [5]:
max_dataset.replace(-99.9, np.nan, inplace=True)
min_dataset.replace(-99.9, np.nan, inplace=True)
avg_dataset.replace(-99.9, np.nan, inplace=True)

Convert from F to C

In [6]:
months_F = column_names[1:]
months_C = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for idx, month in enumerate(months_C):
  max_dataset[month] = ((max_dataset[months_F[idx]] - 32) * 5 / 9).round(1)
  min_dataset[month] = ((min_dataset[months_F[idx]] - 32) * 5 / 9).round(1)
  avg_dataset[month] = ((avg_dataset[months_F[idx]] - 32) * 5 / 9).round(1)

Extract *state_code* and *year* from *Code*

In [7]:
max_dataset['state_code'] = max_dataset['Code'].str[:3]
max_dataset['year'] = max_dataset['Code'].str[6:]

min_dataset['state_code'] = min_dataset['Code'].str[:3]
min_dataset['year'] = min_dataset['Code'].str[6:]

avg_dataset['state_code'] = avg_dataset['Code'].str[:3]
avg_dataset['year'] = avg_dataset['Code'].str[6:]

Create a dictionary with *state_code* as key and its corresponding *state* nomenclature as value

In [8]:
data = """001 Alabama
002 Arizona
003 Arkansas
004 California
005 Colorado
006 Connecticut
007 Delaware
008 Florida
009 Georgia
010 Idaho
011 Illinois
012 Indiana
013 Iowa
014 Kansas
015 Kentucky
016 Louisiana
017 Maine
018 Maryland
019 Massachusetts
020 Michigan
021 Minnesota
022 Mississippi
023 Missouri
024 Montana
025 Nebraska
026 Nevada
027 New Hampshire
028 New Jersey
029 New Mexico
030 New York
031 North Carolina
032 North Dakota
033 Ohio
034 Oklahoma
035 Oregon
036 Pennsylvania
037 Rhode Island
038 South Carolina
039 South Dakota
040 Tennessee
041 Texas
042 Utah
043 Vermont
044 Virginia
045 Washington
046 West Virginia
047 Wisconsin
048 Wyoming
050 Alaska
101 Northeast Region
102 East North Central Region
103 Central Region
104 Southeast Region
105 West North Central Region
106 South Region
107 Southwest Region
108 Northwest Region
109 West Region
110 National (contiguous 48 States)
111 Great Plains
115 Southern Plains and Gulf Coast
120 US Rockies and Westward
121 NWS Eastern Region
122 NWS Southern Region
123 NWS Central Region
124 NWS Western Region
201 Pacific Northwest Basin
202 California River Basin
203 Great Basin
204 Lower Colorado River Basin
205 Upper Colorado River Basin
206 Rio Grande River Basin
207 Texas Gulf Coast River Basin
208 Arkansas-White-Red Basin
209 Lower Mississippi River Basin
210 Missouri River Basin
211 Souris-Red-Rainy Basin
212 Upper Mississippi River Basin
213 Great Lakes Basin
214 Tennessee River Basin
215 Ohio River Basin
216 South Atlantic-Gulf Basin
217 Mid-Atlantic Basin
218 New England Basin
220 Mississippi River Basin & Tributaties (N. of Memphis, TN)
250 Spring Wheat Belt (area weighted)
255 Primary Hard Red Winter Wheat Belt (area weighted)
256 Winter Wheat Belt (area weighted)
260 Primary Corn and Soybean Belt (area weighted)
261 Corn Belt (area weighted)
262 Soybean Belt (area weighted)
265 Cotton Belt (area weighted)
350 Spring Wheat Belt (productivity weighted)
356 Winter Wheat Belt (productivity weighted)
361 Corn Belt (productivity weighted)
362 Soybean Belt (productivity weighted)
365 Cotton Belt (productivity weighted)
450 Spring Wheat Belt (% productivity in the Palmer Z Index)
456 Winter Wheat Belt (% productivity in the Palmer Z Index)
461 Corn Belt (% productivity in the Palmer Z Index)
462 Soybean Belt (% productivity in the Palmer Z Index)
465 Cotton Belt (% productivity in the Palmer Z Index)"""

lines = data.split('\n')
code_to_state = {line[:3]: line[4:].strip() for line in lines}
print(code_to_state)

{'001': 'Alabama', '002': 'Arizona', '003': 'Arkansas', '004': 'California', '005': 'Colorado', '006': 'Connecticut', '007': 'Delaware', '008': 'Florida', '009': 'Georgia', '010': 'Idaho', '011': 'Illinois', '012': 'Indiana', '013': 'Iowa', '014': 'Kansas', '015': 'Kentucky', '016': 'Louisiana', '017': 'Maine', '018': 'Maryland', '019': 'Massachusetts', '020': 'Michigan', '021': 'Minnesota', '022': 'Mississippi', '023': 'Missouri', '024': 'Montana', '025': 'Nebraska', '026': 'Nevada', '027': 'New Hampshire', '028': 'New Jersey', '029': 'New Mexico', '030': 'New York', '031': 'North Carolina', '032': 'North Dakota', '033': 'Ohio', '034': 'Oklahoma', '035': 'Oregon', '036': 'Pennsylvania', '037': 'Rhode Island', '038': 'South Carolina', '039': 'South Dakota', '040': 'Tennessee', '041': 'Texas', '042': 'Utah', '043': 'Vermont', '044': 'Virginia', '045': 'Washington', '046': 'West Virginia', '047': 'Wisconsin', '048': 'Wyoming', '050': 'Alaska', '101': 'Northeast Region', '102': 'East Nort

Add column *state* to the dataset using the dictionary

In [9]:
max_dataset['state'] = max_dataset.apply(lambda row: code_to_state[row['state_code']]
                                         if row['state_code'] in code_to_state
                                         else np.nan, axis=1)

min_dataset['state'] = min_dataset.apply(lambda row: code_to_state[row['state_code']]
                                         if row['state_code'] in code_to_state
                                         else np.nan, axis=1)

avg_dataset['state'] = avg_dataset.apply(lambda row: code_to_state[row['state_code']]
                                         if row['state_code'] in code_to_state
                                         else np.nan, axis=1)

Drop useless columns

In [10]:
max_dataset.drop(columns = ['Code', 'state_code'], inplace = True)
min_dataset.drop(columns = ['Code', 'state_code'], inplace = True)
avg_dataset.drop(columns = ['Code', 'state_code'], inplace = True)

Min and Max per some states

In [11]:
arizona_min = min_dataset[min_dataset['state'] == 'Arizona']
print(min((arizona_min.iloc[:, 12:24]).min()))
arizona_max = max_dataset[max_dataset['state'] == 'Arizona']
print(max((arizona_max.iloc[:, 12:24]).max()))

alabama_min = min_dataset[min_dataset['state'] == 'Alabama']
print(min((alabama_min.iloc[:, 12:24]).min()))
alabama_max = max_dataset[max_dataset['state'] == 'Alabama']
print(max((alabama_max.iloc[:, 12:24]).max()))

-8.5
38.3
-6.0
36.3


In [12]:
arizona_min

,JanF,FebF,MarF,AprF,MayF,JunF,JulF,AugF,SepF,OctF,...,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,year,state
129,31.8,31.0,35.4,42.6,50.9,55.9,65.6,65.5,58.2,48.6,...,10.5,13.3,18.7,18.6,14.6,9.2,1.8,-3.4,1895,Arizona
130,32.5,32.5,38.0,40.9,51.3,61.9,67.2,66.0,60.7,50.1,...,10.7,16.6,19.6,18.9,15.9,10.1,2.7,-0.7,1896,Arizona
131,32.4,30.5,33.0,41.9,52.1,56.7,65.4,66.5,60.9,46.8,...,11.2,13.7,18.6,19.2,16.1,8.2,2.1,-3.8,1897,Arizona
132,24.3,33.0,33.8,47.2,49.3,60.3,68.7,68.1,58.5,44.8,...,9.6,15.7,20.4,20.1,14.7,7.1,0.8,-4.3,1898,Arizona
133,25.9,30.4,35.7,42.9,44.8,59.4,67.8,62.6,59.6,45.9,...,7.1,15.2,19.9,17.0,15.3,7.7,3.0,-1.3,1899,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,30.7,29.4,38.5,45.1,46.2,58.6,67.0,67.2,59.9,43.9,...,7.9,14.8,19.4,19.6,15.5,6.6,3.4,-0.2,2019,Arizona
254,30.7,32.0,38.8,44.0,53.5,60.0,67.8,68.8,59.3,48.8,...,11.9,15.6,19.9,20.4,15.2,9.3,3.8,-2.4,2020,Arizona
255,29.4,32.4,35.3,45.5,51.2,64.3,68.7,66.2,60.8,45.4,...,10.7,17.9,20.4,19.0,16.0,7.4,4.8,0.9,2021,Arizona
256,30.2,29.4,36.9,44.6,51.9,63.0,68.6,66.3,62.1,48.1,...,11.1,17.2,20.3,19.1,16.7,8.9,0.1,-0.4,2022,Arizona


## Export the .csv files

In [13]:
def create_csv(csv_name, dataset):

  '''
  csv_name = output filename
  dataset = temperature dataset from which data has to be read row by row
  '''

  with open(csv_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    header = ['state', 'year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'JanF', 'FebF', 'MarF', 'AprF', 'MayF', 'JunF', 'JulF', 'AugF', 'SepF', 'OctF', 'NovF', 'DecF']
    writer.writerow(header)
    for idx in range(dataset.shape[0]):
      # row with: state, year, 12 months temperatures in C, 12 months temperatures in F
      row = [dataset.iloc[idx, 25]] + [int(dataset.iloc[idx, 24])] + list(dataset.iloc[idx, 12:24]) + list(dataset.iloc[idx, :12])
      writer.writerow(row)

In [14]:
state_list = list(code_to_state.values())[:49]

# Create .csv files with MAX temperatures for each state
for state in state_list:
  csv_name = str(state.replace(" ", "")) + 'MAX.csv'
  dataset = max_dataset[max_dataset['state'] == state]
  create_csv(csv_name, dataset)

# Create .csv files with MIN temperatures for each state
for state in state_list:
  csv_name = str(state.replace(" ", "")) + 'MIN.csv'
  dataset = min_dataset[min_dataset['state'] == state]
  create_csv(csv_name, dataset)

# Create .csv files with AVG temperatures for each state
for state in state_list:
  csv_name = str(state.replace(" ", "")) + 'AVG.csv'
  dataset = avg_dataset[avg_dataset['state'] == state]
  create_csv(csv_name, dataset)

# Load tree_dataset

Uncomment the following line of code to load the already pre-processed *tree_dataset* from google drive

In [15]:
tree_dataset = pd.read_csv("/content/drive/MyDrive/tree_dataset/final_dataset/tree_dataset.csv", low_memory=False)

# Prepare data for mapping

In [16]:
def create_abundance_csv(csv_name, header, name, abundance):

  with open(csv_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for idx in range(len(state)):
      row = [name[idx], abundance[idx]]
      writer.writerow(row)

In [17]:
def create_abundance_fips_csv(csv_name, header, name, abundance, fips_codes):

  with open(csv_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for idx in range(len(state)):
      row = [name[idx], abundance[idx], fips_codes.get(name[idx])]
      writer.writerow(row)

## State abundance

In [18]:
csv_name = "state_abundance.csv"
header = ['state', 'abundance']
state = list(tree_dataset['state'].value_counts().index)
abundance = list(tree_dataset['state'].value_counts().values)
create_abundance_csv(csv_name, header, state, abundance)

## State abundance and fips code

In [19]:
fips_codes = {
    "Alabama": "US01",
    "Alaska": "US02",
    "Arizona": "US04",
    "Arkansas": "US05",
    "California": "US06",
    "Colorado": "US08",
    "Connecticut": "US09",
    "Delaware": "US10",
    "District of Columbia": "US11",
    "Florida": "US12",
    "Georgia": "US13",
    "Hawaii": "US15",
    "Idaho": "US16",
    "Illinois": "US17",
    "Indiana": "US18",
    "Iowa": "US19",
    "Kansas": "US20",
    "Kentucky": "US21",
    "Louisiana": "US22",
    "Maine": "US23",
    "Maryland": "US24",
    "Massachussetts": "US25",
    "Michigan": "US26",
    "Minnesota": "US27",
    "Mississippi": "US28",
    "Missouri": "US29",
    "Montana": "US30",
    "Nebraska": "US31",
    "Nevada": "US32",
    "New Hampshire": "US33",
    "New Jersey": "US34",
    "New Mexico": "US35",
    "New York": "US36",
    "North Carolina": "US37",
    "North Dakota": "US38",
    "Ohio": "US39",
    "Oklahoma": "US40",
    "Oregon": "US41",
    "Pennsylvania": "US42",
    "Rhode Island": "US44",
    "South Carolina": "US45",
    "South Dakota": "US46",
    "Tennessee": "US47",
    "Texas": "US48",
    "Utah": "US49",
    "Vermont": "US50",
    "Virginia": "US51",
    "Washington": "US53",
    "West Virginia": "US54",
    "Wisconsin": "US55",
    "Wyoming": "US56"
}

In [20]:
csv_name = "state_abundance_fips.csv"
header = ['state', 'abundance', 'fips']
state = list(tree_dataset['state'].value_counts().index)
abundance = list(tree_dataset['state'].value_counts().values)
create_abundance_fips_csv(csv_name, header, state, abundance, fips_codes)

## State abundance and postal

In [21]:
postal_codes = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Puerto Rico": "PR",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Virgin Islands": "VI",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

In [22]:
csv_name = "state_abundance_postal.csv"
header = ['state', 'abundance', 'postal']
state = list(tree_dataset['state'].value_counts().index)
abundance = list(tree_dataset['state'].value_counts().values)
create_abundance_fips_csv(csv_name, header, state, abundance, postal_codes)

## City abundance

In [23]:
csv_name = "city_abundance.csv"
header = ['city', 'abundance']
city = list(tree_dataset['greater_metro'].value_counts().index)
abundance = list(tree_dataset['greater_metro'].value_counts().values)
create_abundance_csv(csv_name, header, city, abundance)

## JSON

In [47]:
abundance_df = pd.read_csv("/content/state_abundance_postal.csv", low_memory=False)
abundance_dict = {}

for idx in range(abundance_df.shape[0]):
  name = list(abundance_df.iloc[idx])[0]
  abundance = list(abundance_df.iloc[idx])[1]
  postal = list(abundance_df.iloc[idx])[2]
  abundance_dict[postal] = [name, abundance]

for key, value in postal_codes.items():
  if abundance_dict.get(value) is None:
    name = key
    abundance = 0
    abundance_dict[value] = [name, abundance]
  else:
    name = abundance_dict.get(value)[0]
    abundance = abundance_dict.get(value)[1]

In [48]:
json_path = "/content/drive/MyDrive/tree_dataset/final_dataset/us-states.json"
with open(json_path, 'r') as file:
    json_file = json.load(file)

In [49]:
for idx in range(len(json_file['objects']['states'].get('geometries'))):
  properties = json_file['objects']['states'].get('geometries')[idx]['properties']
  postal = properties['postal']
  try:
    name, abundance = abundance_dict[postal]
    properties['name'] = name
    properties['abundance'] = str(abundance)
    print(properties)
  except:
    pass

{'postal': 'AK', 'name': 'Alaska', 'abundance': '0'}
{'postal': 'AL', 'name': 'Alabama', 'abundance': '0'}
{'postal': 'AR', 'name': 'Arkansas', 'abundance': '0'}
{'postal': 'AZ', 'name': 'Arizona', 'abundance': '214'}
{'postal': 'CA', 'name': 'California', 'abundance': '1806085'}
{'postal': 'CO', 'name': 'Colorado', 'abundance': '354878'}
{'postal': 'CT', 'name': 'Connecticut', 'abundance': '0'}
{'postal': 'DC', 'name': 'District of Columbia', 'abundance': '175847'}
{'postal': 'DE', 'name': 'Delaware', 'abundance': '0'}
{'postal': 'FL', 'name': 'Florida', 'abundance': '124208'}
{'postal': 'GA', 'name': 'Georgia', 'abundance': '41175'}
{'postal': 'HI', 'name': 'Hawaii', 'abundance': '15150'}
{'postal': 'IA', 'name': 'Iowa', 'abundance': '15805'}
{'postal': 'ID', 'name': 'Idaho', 'abundance': '0'}
{'postal': 'IL', 'name': 'Illinois', 'abundance': '0'}
{'postal': 'IN', 'name': 'Indiana', 'abundance': '97227'}
{'postal': 'KS', 'name': 'Kansas', 'abundance': '44767'}
{'postal': 'KY', 'name'

In [50]:
out_path = "/content/choropleth.json"
with open(out_path, 'w') as file:
    json.dump(json_file, file)

## Drop data with no longitude/latitude

In [ ]:
# tree_dataset.dropna(subset=['longitude_coordinate', 'latitude_coordinate'], how='any', inplace=True)

# Download ALL .csv files

In [ ]:
directory_path = '/content'
file_type = ".csv"
files_to_download = glob.glob(f"{directory_path}/*{file_type}")
zip_filename = "datasets.zip"
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in files_to_download:
        zipf.write(file, arcname=os.path.basename(file))
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>